In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import csv
import tensorflow as tf
import timeit
import seaborn as sns
import random
import math
from tensorflow.keras.utils import to_categorical
from pandas import DataFrame
from copy import copy, deepcopy
from beautifultable import BeautifulTable

In [2]:
class Person:
    def __init__(self, id, filename, img, gender, ageGroup, age):
        self.id = id
        self.filename = filename
        self.img = img
        self.gender = gender
        self.ageGroup = ageGroup
        self.age = age
        self.guessGender = 0
        self.guessAgeGroup = 0
        self.guessAge = 0

In [3]:
%matplotlib inline

df=pd.read_csv('Age_gender_info.csv')
id_train = np.loadtxt("GEI_IDList_train.txt", dtype='int32')



In [4]:
df = df.sort_values(by=['ID']).reset_index(drop=True)

In [5]:
df

,ID,Age,Gender
0,1,52,F
1,28,32,M
2,30,38,M
3,31,64,M
4,33,44,F
...,...,...,...
63841,95100,28,F
63842,95105,45,F
63843,95106,9,F
63844,95107,11,M


In [18]:
datadir = r"C:/Users/Acer/FYP/GEI/"
IMG_SIZE = 80
all_people = []
df.ID = df.ID.astype(str)

for index, data in df.iterrows():
    id = data.ID
    filename = os.path.join(datadir, '0'*(8-len(id)) + id + '.png')
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    
    gender = 0 if data.Gender == 'F' else 1
    
    age = data.Age
    if age < 6:
        age = 0
    elif age < 11:
        age = 1
    elif age < 16:
        age = 2
    elif age < 61:
        age = 3
    else:
        age = 4
    
    p = Person(id, filename, img, gender, age)
    all_people.append(p)


In [17]:
people_train = []
people_test = []

for p in all_people:
    if int(p.id) in id_train:
        people_train.append(p)
    else:
        people_test.append(p)

In [15]:
block_gender = tf.keras.models.load_model('model_OULP-Age/OULP-Age_gender.h5')

block_F_age_group = tf.keras.models.load_model('model_OULP-Age/OULP-Age_F_age_group.h5')
block_M_age_group = tf.keras.models.load_model('model_OULP-Age/OULP-Age_M_age_group.h5')

block_F_group0 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_F_group0.h5')
block_F_group1 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_F_group1.h5')
block_F_group2 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_F_group2.h5')
block_F_group3 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_F_group3.h5')
block_F_group4 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_F_group4.h5')

block_M_group0 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_M_group0.h5')
block_M_group1 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_M_group1.h5')
block_M_group2 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_M_group2.h5')
block_M_group3 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_M_group3.h5')
block_M_group4 = tf.keras.models.load_model('model_OULP-Age/OULP-Age_M_group4.h5')

In [16]:
model = {'model':block_gender}

model_F = {'model':block_F_age_group, 0:block_F_group0, 1:block_F_group1, 2:block_F_group2, 3:block_F_group3, 4:block_F_group4}
model_M = {'model':block_M_age_group, 0:block_M_group0, 1:block_M_group1, 2:block_M_group2, 3:block_M_group3, 4:block_M_group4}

model.update({0:model_F, 1:model_M})


In [49]:
category_gender = ["F","M"]
category_ageGroup = ["0-5","6-10","11-15","16-60","Over 60"]

In [ ]:
start = timeit.default_timer()
for p in all_people:
    img = np.array(p.img).reshape(-1, IMG_SIZE,IMG_SIZE,1)
    
    predict_gender = model['model'].predict(img)
    p.guessGender = np.argmax(predict_gender)
    
    predict_ageGroup = model[category_gender[p.guessGender]].predict(img)
    p.guessAgeGroup = np.argmax(predict_ageGroup)
    
    predict_age = model[category_gender[p.guessGender]][category_ageGroup[p.guessAgeGroup]].predict(img)
    p.guessAge = np.argmax(predict_age)
        
stop = timeit.default_timer()
    

In [25]:
s = stop - start
hours = ((stop - start)/60)/60
mins = hours*60 - round(hours)*60

print('Prediction Time: ', round(hours), 'hours', round(mins), 'mins')

Prediction Time:  1 hours 1 mins


# Saving results to a .csv file

In [32]:
col = ['id', 'filename', 'gender', 'age', 'guessGender', 'guessAge']
pred_prople = []
for p in all_people:
    filename = p.filename.split('/FYP/')[1]
    pred_prople.append([p.id,filename,p.gender,p.age,p.guessGender,p.guessAge])

df_pred_prople = DataFrame(pred_prople, columns = col)
df_pred_prople.to_csv('C:/Users/Acer/FYP/Predicted_Result.csv',index=0)

In [34]:
df_pred_prople.to_csv('C:/Users/Acer/FYP/Predicted_Result.csv',index=0)

# Truth table of gender

In [19]:
truth_table_gender = {}
df_gender = []
total = [0] * (len(category_gender) + 1)

for c in category_gender:
    truth_table_gender[c] = {}
    for c2 in category_gender:
        truth_table_gender[c][c2] = 0
        
for p in people_test:
    truth_table_gender[category_gender[p.guessGender]][category_gender[p.gender]]+=1

for gender in truth_table_gender:
    temp = []
    index = 0
    count = 0
    for key in truth_table_gender[gender]:
        temp.append(truth_table_gender[gender][key])
        count+=truth_table_gender[gender][key]
        total[index]+=truth_table_gender[gender][key]
        index+=1
    total[len(total) - 1]+=count
    temp.append(count)
    df_gender.append(temp)
df_gender.append(total)

In [20]:
df_gender

[[15957, 356, 16313], [469, 15141, 15610], [16426, 15497, 31923]]

In [21]:
columns = deepcopy(category_gender)
columns.insert(0,'Truth -->\n\nPredict')
columns.append('Total')
for i in range(0, len(columns)):
    columns[i] = str(columns[i])

table = BeautifulTable()
table.columns.header = columns
for i in range(0,len(df_gender)):
    df_gender[i].insert(0,columns[i+1])
    table.rows.append(df_gender[i])

table.maxwidth = 150
print(table)

+-----------+-------+-------+-------+
| Truth --> |   F   |   M   | Total |
|           |       |       |       |
|  Predict  |       |       |       |
+-----------+-------+-------+-------+
|     F     | 15957 |  356  | 16313 |
+-----------+-------+-------+-------+
|     M     |  469  | 15141 | 15610 |
+-----------+-------+-------+-------+
|   Total   | 16426 | 15497 | 31923 |
+-----------+-------+-------+-------+


In [22]:
table = BeautifulTable()
table.columns.header = ['Truth -->\n\nPredict','F','M','Total']
table.rows.append(['F',15704,750,16454])
table.rows.append(['M',722,14747,15469])
table.rows.append(['Total',16426,15497,31923])

table.maxwidth = 150
print(table)

+-----------+-------+-------+-------+
| Truth --> |   F   |   M   | Total |
|           |       |       |       |
|  Predict  |       |       |       |
+-----------+-------+-------+-------+
|     F     | 15704 |  750  | 16454 |
+-----------+-------+-------+-------+
|     M     |  722  | 14747 | 15469 |
+-----------+-------+-------+-------+
|   Total   | 16426 | 15497 | 31923 |
+-----------+-------+-------+-------+


# Truth table of age group

In [53]:
truth_table_ageGroup = {}
df_ageGroup = []
total = [0] * (len(category_ageGroup) + 1)

for c in category_ageGroup:
    truth_table_ageGroup[c] = {}
    for c2 in category_ageGroup:
        truth_table_ageGroup[c][c2] = 0
        
for p in all_people:
    if p.age == '-':
        continue
    truth_table_ageGroup[category_ageGroup[p.guessAgeGroup]][category_ageGroup[p.ageGroup]]+=1

for ageGroup in truth_table_ageGroup:
    temp = []
    index = 0
    count = 0
    for key in truth_table_ageGroup[ageGroup]:
        temp.append(truth_table_ageGroup[ageGroup][key])
        count+=truth_table_ageGroup[ageGroup][key]
        total[index]+=truth_table_ageGroup[ageGroup][key]
        index+=1
    total[len(total) - 1]+=count
    temp.append(count)
    df_ageGroup.append(temp)
df_ageGroup.append(total)

In [54]:
columns = deepcopy(category_ageGroup)
columns.insert(0,'Truth -->\n\nPredict')
columns.append('Total')
for i in range(0, len(columns)):
    columns[i] = str(columns[i])

table = BeautifulTable()
table.columns.header = columns
for i in range(0,len(df_ageGroup)):
    df_ageGroup[i].insert(0,columns[i+1])
    table.rows.append(df_ageGroup[i])

table.maxwidth = 150

print('Male')
print(table)

Male
+-----------+------+------+-------+-------+---------+-------+
| Truth --> | 0-5  | 6-10 | 11-15 | 16-60 | Over 60 | Total |
|           |      |      |       |       |         |       |
|  Predict  |      |      |       |       |         |       |
+-----------+------+------+-------+-------+---------+-------+
|    0-5    | 1315 | 248  |   3   |   2   |    2    | 1570  |
+-----------+------+------+-------+-------+---------+-------+
|   6-10    | 245  | 8571 |  611  |  72   |   15    | 9514  |
+-----------+------+------+-------+-------+---------+-------+
|   11-15   |  4   | 633  | 6691  |  842  |   50    | 8220  |
+-----------+------+------+-------+-------+---------+-------+
|   16-60   |  10  | 197  | 1401  | 41031 |   605   | 43244 |
+-----------+------+------+-------+-------+---------+-------+
|  Over 60  |  0   |  5   |  18   |  163  |  1112   | 1298  |
+-----------+------+------+-------+-------+---------+-------+
|   Total   | 1574 | 9654 | 8724  | 42110 |  1784   | 63846 |
+--

In [37]:
truth_table_ageGroup = {}
df_ageGroup = []
total = [0] * (len(category_ageGroup) + 1)

for c in category_ageGroup:
    truth_table_ageGroup[c] = {}
    for c2 in category_ageGroup:
        truth_table_ageGroup[c][c2] = 0
        
for p in people_test:
    if p.age == '-' or p.guessGender == 0:
        continue
    truth_table_ageGroup[category_ageGroup[p.guessAgeGroup]][category_ageGroup[p.ageGroup]]+=1

for ageGroup in truth_table_ageGroup:
    temp = []
    index = 0
    count = 0
    for key in truth_table_ageGroup[ageGroup]:
        temp.append(truth_table_ageGroup[ageGroup][key])
        count+=truth_table_ageGroup[ageGroup][key]
        total[index]+=truth_table_ageGroup[ageGroup][key]
        index+=1
    total[len(total) - 1]+=count
    temp.append(count)
    df_ageGroup.append(temp)
df_ageGroup.append(total)

In [38]:
columns = deepcopy(category_ageGroup)
columns.insert(0,'Truth -->\n\nPredict')
columns.append('Total')
for i in range(0, len(columns)):
    columns[i] = str(columns[i])

table = BeautifulTable()
table.columns.header = columns
for i in range(0,len(df_ageGroup)):
    df_ageGroup[i].insert(0,columns[i+1])
    table.rows.append(df_ageGroup[i])

table.maxwidth = 150

print(table)

+-----------+-----+------+-------+-------+---------+-------+
| Truth --> | 0-5 | 6-10 | 11-15 | 16-60 | Over 60 | Total |
|           |     |      |       |       |         |       |
|  Predict  |     |      |       |       |         |       |
+-----------+-----+------+-------+-------+---------+-------+
|    0-5    | 210 | 116  |   1   |   0   |    1    |  328  |
+-----------+-----+------+-------+-------+---------+-------+
|   6-10    | 98  | 1839 |  352  |  19   |   13    | 2321  |
+-----------+-----+------+-------+-------+---------+-------+
|   11-15   |  0  | 329  | 1232  |  356  |   42    | 1959  |
+-----------+-----+------+-------+-------+---------+-------+
|   16-60   |  0  |  66  |  577  | 9783  |   337   | 10763 |
+-----------+-----+------+-------+-------+---------+-------+
|  Over 60  |  0  |  3   |   8   |  84   |   144   |  239  |
+-----------+-----+------+-------+-------+---------+-------+
|   Total   | 308 | 2353 | 2170  | 10242 |   537   | 15610 |
+-----------+-----+-----

In [34]:
truth_table_ageGroup = {}
df_ageGroup = []
total = [0] * (len(category_ageGroup) + 1)

for c in category_ageGroup:
    truth_table_ageGroup[c] = {}
    for c2 in category_ageGroup:
        truth_table_ageGroup[c][c2] = 0
        
for p in people_test:
    truth_table_ageGroup[category_ageGroup[p.guessAgeGroup]][category_ageGroup[p.ageGroup]]+=1

for ageGroup in truth_table_ageGroup:
    temp = []
    index = 0
    count = 0
    for key in truth_table_ageGroup[ageGroup]:
        temp.append(truth_table_ageGroup[ageGroup][key])
        count+=truth_table_ageGroup[ageGroup][key]
        total[index]+=truth_table_ageGroup[ageGroup][key]
        index+=1
    total[len(total) - 1]+=count
    temp.append(count)
    df_ageGroup.append(temp)
df_ageGroup.append(total)

In [35]:
columns = deepcopy(category_ageGroup)
columns.insert(0,'Truth -->\n\nPredict')
columns.append('Total')
for i in range(0, len(columns)):
    columns[i] = str(columns[i])

table = BeautifulTable()
table.columns.header = columns
for i in range(0,len(df_ageGroup)):
    df_ageGroup[i].insert(0,columns[i+1])
    table.rows.append(df_ageGroup[i])

table.maxwidth = 150

print(table)

+-----------+-----+------+-------+-------+---------+-------+
| Truth --> | 0-5 | 6-10 | 11-15 | 16-60 | Over 60 | Total |
|           |     |      |       |       |         |       |
|  Predict  |     |      |       |       |         |       |
+-----------+-----+------+-------+-------+---------+-------+
|    0-5    | 491 | 248  |   3   |   1   |    1    |  744  |
+-----------+-----+------+-------+-------+---------+-------+
|   6-10    | 241 | 3833 |  610  |  72   |   15    | 4771  |
+-----------+-----+------+-------+-------+---------+-------+
|   11-15   |  4  | 633  | 2320  |  842  |   50    | 3849  |
+-----------+-----+------+-------+-------+---------+-------+
|   16-60   |  8  | 195  | 1400  | 19890 |   604   | 22097 |
+-----------+-----+------+-------+-------+---------+-------+
|  Over 60  |  0  |  5   |  18   |  162  |   277   |  462  |
+-----------+-----+------+-------+-------+---------+-------+
|   Total   | 744 | 4914 | 4351  | 20967 |   947   | 31923 |
+-----------+-----+-----

In [39]:
table = BeautifulTable()
table.columns.header = ['Truth -->\n\nPredict','0-5','6-10','11-15', '16-60','Over 60','Total']
table.rows.append(['0-5',266,212,1,1,0,480])
table.rows.append(['6-10',36,1912,350,4,9,2311])
table.rows.append(['11-15',0,213,1209,120,15,1557])
table.rows.append(['16-60',0,17,604,9969,244,10834])
table.rows.append(['Over 60 ',0,0,9,137,266,412])
table.rows.append(['Total',305,2354,2173,10231,534,15594])

table.maxwidth = 150
print(table)

+-----------+-----+------+-------+-------+---------+-------+
| Truth --> | 0-5 | 6-10 | 11-15 | 16-60 | Over 60 | Total |
|           |     |      |       |       |         |       |
|  Predict  |     |      |       |       |         |       |
+-----------+-----+------+-------+-------+---------+-------+
|    0-5    | 266 | 212  |   1   |   1   |    0    |  480  |
+-----------+-----+------+-------+-------+---------+-------+
|   6-10    | 36  | 1912 |  350  |   4   |    9    | 2311  |
+-----------+-----+------+-------+-------+---------+-------+
|   11-15   |  0  | 213  | 1209  |  120  |   15    | 1557  |
+-----------+-----+------+-------+-------+---------+-------+
|   16-60   |  0  |  17  |  604  | 9969  |   244   | 10834 |
+-----------+-----+------+-------+-------+---------+-------+
| Over 60   |  0  |  0   |   9   |  137  |   266   |  412  |
+-----------+-----+------+-------+-------+---------+-------+
|   Total   | 305 | 2354 | 2173  | 10231 |   534   | 15594 |
+-----------+-----+-----

# Other table

In [11]:
young = []
old = []
for p in all_people:
    if p.age <= 60:
        young.append(p)
    else:
        old.append(p)
    

In [29]:
TY = 0
FY = 0
TO = 0
FO = 0

for p in young:
    if p.guessAge <= 60:
        TY+=1
    else:
        FY+=1
        
for p in old:
    if p.guessAge > 60:
        TO+=1
    else:
        FO+=1


In [32]:
RecallYoung = TY/(TY+FO)
RecallElder = TO/(TO+FY)
PrecisionYoung = TY/(TY+FY)
PrecisionElder = TO/(TO+FO)
CCR = (TY+TO)/(TO+FO+TY+FY)

In [46]:
print(
    " RecallYoung:",round(RecallYoung*100,2),'%\n',
    "RecallElder:",round(RecallElder*100,2),'%\n',
    "PrecisionYoung:",round(PrecisionYoung*100,2),'%\n',
    "PrecisionElder:",round(PrecisionElder*100,2),'%\n',
    "CCR:",round(CCR*100,2),'%')

 RecallYoung: 98.93 %
 RecallElder: 85.67 %
 PrecisionYoung: 99.7 %
 PrecisionElder: 62.33 %
 CCR: 98.66 %


# Eveluate age

In [24]:
MAE = {}
MAE['age group']={}
for c in category_ageGroup:
    MAE['age group'][c]=0
MAE

{'age group': {'0-5': 0, '6-10': 0, '11-15': 0, '16-60': 0, 'Over 60': 0}}

In [25]:
length = [0,0,0,0,0]
mean = 0

for p in all_people:
    MAE['age group'][category_ageGroup[p.ageGroup]]+=abs(p.age-p.guessAge)
    length[p.ageGroup]+=1
    
for group in MAE['age group']:
    mean += MAE['age group'][group]
    MAE['age group'][group] = MAE['age group'][group]/length[p.ageGroup]

MAE['age group']['mean'] = mean/len(all_people)

for group in MAE['age group']:
    MAE['age group'][group] = round(MAE['age group'][group],2)


In [26]:
Metrics = []
for group in MAE['age group']:
    Metrics.append(MAE['age group'][group])

columns = deepcopy(category_ageGroup)
columns.append('Mean')
for i in range(0, len(columns)):
    columns[i] = str(columns[i])

table = BeautifulTable()
table.columns.header = columns
table.rows.append(Metrics)
table.rows.header = ["MAE"]
table.maxwidth = 150
print(table)

+-----+------+------+-------+-------+---------+------+
|     | 0-5  | 6-10 | 11-15 | 16-60 | Over 60 | Mean |
+-----+------+------+-------+-------+---------+------+
| MAE | 0.02 | 0.21 | 0.35  | 8.81  |   0.6   | 6.6  |
+-----+------+------+-------+-------+---------+------+


In [27]:
MAE['gender']={}
for c in category_gender:
    MAE['gender'][c]=0
MAE

{'age group': {'0-5': 0.02,
  '6-10': 0.21,
  '11-15': 0.35,
  '16-60': 8.81,
  'Over 60': 0.6,
  'mean': 6.6},
 'gender': {'F': 0, 'M': 0}}

In [28]:
length = [0,0]
mean = 0

for p in all_people:
    MAE['gender'][category_gender[p.gender]]+=abs(p.age-p.guessAge)
    length[p.gender]+=1
    
for group in MAE['gender']:
    mean += MAE['gender'][group]
    MAE['gender'][group] = MAE['gender'][group]/length[p.gender]

MAE['gender']['mean'] = mean/len(all_people)

for group in MAE['gender']:
    MAE['gender'][group] = round(MAE['gender'][group],2)

In [29]:
Metrics = []
for group in MAE['gender']:
    Metrics.append(MAE['gender'][group])

columns = deepcopy(category_gender)
columns.append('Mean')
for i in range(0, len(columns)):
    columns[i] = str(columns[i])

table = BeautifulTable()
table.columns.header = columns
table.rows.append(Metrics)
table.rows.header = ["MAE"]
table.maxwidth = 150
print(table)

+-----+------+------+------+
|     |  F   |  M   | Mean |
+-----+------+------+------+
| MAE | 6.73 | 6.13 | 6.6  |
+-----+------+------+------+


In [30]:
table = BeautifulTable()
table.columns.header = ['','F','M','Mean']
table.rows.append(['MAE',6.07,5.6,5.84])

table.maxwidth = 150
print(table)

+-----+------+-----+------+
|     |  F   |  M  | Mean |
+-----+------+-----+------+
| MAE | 6.07 | 5.6 | 5.84 |
+-----+------+-----+------+


In [31]:
table = BeautifulTable()
table.columns.header = ['MAE','0','15','30','45','60','75','90','180','195','210','225','240','255','270','Mean']
table.rows.append(["Paper's Method",8.91,8.83,8.27,8.48,8.35,8.08,7.83,9.24,8.86,8.61,8.48,8.11,7.88,7.75,8.39])
table.rows.append(["Our Method",9.97,9.94,6.34,6.34,6.39,6.09,6.21,10.52,9.85,6.32,6.26,6.5,6.34,6.47,7.4])

table.maxwidth = 150
print(table)

+----------------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+
|      MAE       |  0   |  15  |  30  |  45  |  60  |  75  |  90  |  180  | 195  | 210  | 225  | 240  | 255  | 270  | Mean |
+----------------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+
| Paper's Method | 8.91 | 8.83 | 8.27 | 8.48 | 8.35 | 8.08 | 7.83 | 9.24  | 8.86 | 8.61 | 8.48 | 8.11 | 7.88 | 7.75 | 8.39 |
+----------------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+
|   Our Method   | 9.97 | 9.94 | 6.34 | 6.34 | 6.39 | 6.09 | 6.21 | 10.52 | 9.85 | 6.32 | 6.26 | 6.5  | 6.34 | 6.47 | 7.4  |
+----------------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+


# Percentage of each results

In [17]:
corr_count_gender = 0
corr_count_ageGroup = 0
two_corr = 0

for p in people_test:
    count = 0
    if p.gender == p.guessGender:
        corr_count_gender+=1
        count+=1
    
    if p.ageGroup == p.guessAgeGroup:
        corr_count_ageGroup+=1
        count+=1
        
    if count == 2:
        two_corr+=1
         
acc_gender = corr_count_gender/len(people_test)
acc_ageGroup = corr_count_ageGroup/len(people_test)
two_corr = two_corr/len(people_test)

avg_acc = (acc_gender+acc_ageGroup)/2


In [18]:
print("Accuracy of Predicting gender is: " + str(round(acc_gender*100,2)) + '%')
print("Accuracy of Predicting age is: " + str(round(acc_ageGroup*100,2)) + '%','\n')

print("Accuracy of Two layers passed at the same time: " + str(round(two_corr*100,2)) + '%')
print("The Average Accuracy: " + str(round(avg_acc*100,2)) + '%')

Accuracy of Predicting gender is: 97.42%
Accuracy of Predicting age is: 83.99% 

Accuracy of Two layers passed at the same time: 82.31%
The Average Accuracy: 90.7%


# Calculate SD

In [39]:
MAE['age group']['mean']

6.6

In [42]:
sumation = 0
for p in all_people:
    sumation+=(abs(p.age-p.guessAge) - MAE['age group']['mean']) ** 2

SD = math.sqrt(sumation/(len(all_people)-1))

In [44]:
print(SD)

8.551075863648194


# =========================

In [4]:
table = BeautifulTable()
table.rows.append(['Layer', 'Output dimension', 'Kernel size', 'Strides','Units'])
table.rows.append(['Conv 2D a', '81', '5x5', '1', '-'])
table.rows.append(['Batch Norm a', '-', '-', '-', '-'])
table.rows.append(['Max Pooling a', '-', '3x3', '2', '-'])
table.rows.append(['Conv 2D b', '45', '7x7', '1', '-'])
table.rows.append(['Batch Norm b', '-', '-', '-', '-'])
table.rows.append(['Max Pooling b', '-', '2x2', '2', '-'])
table.rows.append(['Flatten Layer', '-', '-', '-', '-'])
table.rows.append(['Densely-Connected', '-', '-', '-', '1024'])
table.rows.append(['Batch Norm c', '-', '-', '-', '-'])
table.rows.append(['Output Layer', '-', '-', '-', '2'])

#table.maxwidth = 150
print(table)

+-------------------+------------------+-------------+---------+-------+
|       Layer       | Output dimension | Kernel size | Strides | Units |
+-------------------+------------------+-------------+---------+-------+
|     Conv 2D a     |        81        |     5x5     |    1    |   -   |
+-------------------+------------------+-------------+---------+-------+
|   Batch Norm a    |        -         |      -      |    -    |   -   |
+-------------------+------------------+-------------+---------+-------+
|   Max Pooling a   |        -         |     3x3     |    2    |   -   |
+-------------------+------------------+-------------+---------+-------+
|     Conv 2D b     |        45        |     7x7     |    1    |   -   |
+-------------------+------------------+-------------+---------+-------+
|   Batch Norm b    |        -         |      -      |    -    |   -   |
+-------------------+------------------+-------------+---------+-------+
|   Max Pooling b   |        -         |     2x2   